Sentiment Analysis using fine tuning

In [1]:
import pandas as pd

books = pd.read_csv("books_with_categories.csv")

In [3]:
## We upload the huggingface transformer to use the fine tuned LLM model

from transformers import pipeline
classifier = pipeline("text-classification",
                      model = "j-hartmann/emotion-english-distilroberta-base",
                      top_k = None,
                      device = "cpu")
classifier("I love this")

Device set to use cpu


[[{'label': 'joy', 'score': 0.9845667481422424},
  {'label': 'surprise', 'score': 0.0049272035248577595},
  {'label': 'sadness', 'score': 0.004531425889581442},
  {'label': 'neutral', 'score': 0.0034752993378788233},
  {'label': 'anger', 'score': 0.0013875310542061925},
  {'label': 'disgust', 'score': 0.0007134023471735418},
  {'label': 'fear', 'score': 0.0003984913637395948}]]

In [4]:
## Testing the classifier
classifier("I hate this")

[[{'label': 'anger', 'score': 0.6354435086250305},
  {'label': 'disgust', 'score': 0.2896920144557953},
  {'label': 'sadness', 'score': 0.046773847192525864},
  {'label': 'neutral', 'score': 0.018368206918239594},
  {'label': 'fear', 'score': 0.00443401699885726},
  {'label': 'surprise', 'score': 0.003445776179432869},
  {'label': 'joy', 'score': 0.001842743600718677}]]

In [5]:
## We could predict the overall emotion of the description or we could use chunks
books.description[0]

'A NOVEL THAT READERS and critics have been eagerly anticipating for over a decade, Gilead is an astonishingly imagined story of remarkable lives. John Ames is a preacher, the son of a preacher and the grandson (both maternal and paternal) of preachers. It’s 1956 in Gilead, Iowa, towards the end of the Reverend Ames’s life, and he is absorbed in recording his family’s story, a legacy for the young son he will never see grow up. Haunted by his grandfather’s presence, John tells of the rift between his grandfather and his father: the elder, an angry visionary who fought for the abolitionist cause, and his son, an ardent pacifist. He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption. Told in John Ames’s joyous, rambling voice that finds beauty, humour and truth in the smallest of life’s details, Gilead is a song of celebration and acceptance of the best and the worst the world ha

In [6]:
## Running the analyzer over the entire description
classifier(books.description[0])

[[{'label': 'fear', 'score': 0.6548417806625366},
  {'label': 'neutral', 'score': 0.1698516607284546},
  {'label': 'sadness', 'score': 0.11640869081020355},
  {'label': 'surprise', 'score': 0.020700668916106224},
  {'label': 'disgust', 'score': 0.019100654870271683},
  {'label': 'joy', 'score': 0.01516133826225996},
  {'label': 'anger', 'score': 0.003935144282877445}]]

We see that running the classifier over the entire description does not return the most accurate information


In [7]:
## Breaking down into chunks and running the classifier through each of the sentence
classifier(books.description[0].split("."))

[[{'label': 'surprise', 'score': 0.7296023964881897},
  {'label': 'neutral', 'score': 0.14038558304309845},
  {'label': 'fear', 'score': 0.06816234439611435},
  {'label': 'joy', 'score': 0.04794265329837799},
  {'label': 'anger', 'score': 0.009156362153589725},
  {'label': 'disgust', 'score': 0.00262847519479692},
  {'label': 'sadness', 'score': 0.0021221647039055824}],
 [{'label': 'neutral', 'score': 0.44937118887901306},
  {'label': 'disgust', 'score': 0.2735905945301056},
  {'label': 'joy', 'score': 0.10908323526382446},
  {'label': 'sadness', 'score': 0.09362746775150299},
  {'label': 'anger', 'score': 0.040478307753801346},
  {'label': 'surprise', 'score': 0.02697017230093479},
  {'label': 'fear', 'score': 0.006879061460494995}],
 [{'label': 'neutral', 'score': 0.6462149620056152},
  {'label': 'sadness', 'score': 0.24273425340652466},
  {'label': 'disgust', 'score': 0.043422725051641464},
  {'label': 'surprise', 'score': 0.02830054610967636},
  {'label': 'joy', 'score': 0.01421148

In [8]:
sentences = books.description[0].split(".")
emotion = classifier(sentences)
emotion[0]

[{'label': 'surprise', 'score': 0.7296023964881897},
 {'label': 'neutral', 'score': 0.14038558304309845},
 {'label': 'fear', 'score': 0.06816234439611435},
 {'label': 'joy', 'score': 0.04794265329837799},
 {'label': 'anger', 'score': 0.009156362153589725},
 {'label': 'disgust', 'score': 0.00262847519479692},
 {'label': 'sadness', 'score': 0.0021221647039055824}]

As we can see, the first sentence is now classified as surprising

In [9]:
## Lets look at the fourth sentence

sentences[4]

' He is troubled, too, by his prodigal namesake, Jack (John Ames) Boughton, his best friend’s lost son who returns to Gilead searching for forgiveness and redemption'

In [10]:
emotion[4]

[{'label': 'sadness', 'score': 0.9671575427055359},
 {'label': 'neutral', 'score': 0.015104142017662525},
 {'label': 'disgust', 'score': 0.006480592768639326},
 {'label': 'fear', 'score': 0.005393987987190485},
 {'label': 'surprise', 'score': 0.0022869387175887823},
 {'label': 'anger', 'score': 0.0018428885377943516},
 {'label': 'joy', 'score': 0.0017338772304356098}]

This makes more sense than passing the whole description through the classifier

Now we can try to maximum emotion probability for each description


In [16]:
## We now try to order them by emotions instead of their scores

sorted(emotion[0], key = lambda x: x["label"])

[{'label': 'anger', 'score': 0.009156362153589725},
 {'label': 'disgust', 'score': 0.00262847519479692},
 {'label': 'fear', 'score': 0.06816234439611435},
 {'label': 'joy', 'score': 0.04794265329837799},
 {'label': 'neutral', 'score': 0.14038558304309845},
 {'label': 'sadness', 'score': 0.0021221647039055824},
 {'label': 'surprise', 'score': 0.7296023964881897}]

In [17]:
## We try to extract the maximum emotion probability for each description

import numpy as np

emotion_label = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
isbns = []
emotion_score = {label: [] for label in emotion_label}

def calculate_max_scores(prediction):
    per_emotion_score = {label: [] for label in emotion_label}
    for prediction in prediction:
        sorted_prediction = sorted(prediction, key = lambda x: x["label"])
        for index, label in enumerate(emotion_label):
            per_emotion_score[label].append(sorted_prediction[index]["score"])
    return {label: np.max(score) for label, score in per_emotion_score.items()}


In [18]:
## Testing for 10 books
for i in range(10):
    isbns.append(books.isbn13[i])
    sentences = books.description[i].split(".")
    prediction = classifier(sentences)
    max_scores = calculate_max_scores(prediction)
    for label in emotion_label:
        emotion_score[label].append(max_scores[label])

In [19]:
emotion_score

{'anger': [np.float64(0.06413379311561584),
  np.float64(0.612618625164032),
  np.float64(0.06413379311561584),
  np.float64(0.35148394107818604),
  np.float64(0.0814124345779419),
  np.float64(0.23222504556179047),
  np.float64(0.538184642791748),
  np.float64(0.06413379311561584),
  np.float64(0.3006698489189148),
  np.float64(0.06413379311561584)],
 'disgust': [np.float64(0.2735905945301056),
  np.float64(0.34828659892082214),
  np.float64(0.10400686413049698),
  np.float64(0.15072254836559296),
  np.float64(0.18449579179286957),
  np.float64(0.7271746397018433),
  np.float64(0.15585486590862274),
  np.float64(0.10400686413049698),
  np.float64(0.2794814109802246),
  np.float64(0.17792640626430511)],
 'fear': [np.float64(0.928168535232544),
  np.float64(0.9425276517868042),
  np.float64(0.9723207950592041),
  np.float64(0.3607061505317688),
  np.float64(0.09504344314336777),
  np.float64(0.051362939178943634),
  np.float64(0.747427225112915),
  np.float64(0.4044969379901886),
  np.f

This is correct so we can apply to all the books


In [21]:
## Running the code for all the books in the dataset
from tqdm import tqdm

emotion_label = ["anger", "disgust", "fear", "joy", "sadness", "surprise"]
isbns = []
emotion_score = {label: [] for label in emotion_label}

for i in tqdm(range(len(books))):
    isbns.append(books.isbn13[i])
    sentences = books.description[i].split(".")
    prediction = classifier(sentences)
    max_scores = calculate_max_scores(prediction)
    for label in emotion_label:
        emotion_score[label].append(max_scores[label])

100%|██████████| 5197/5197 [09:00<00:00,  9.62it/s]


In [22]:
## We make a dataframe with the scores
emotions_df = pd.DataFrame(emotion_score)
emotions_df["isbn13"] = isbns

In [25]:
## Merge back to the books dataframe
books = pd.merge(books, emotions_df, on = "isbn13")
books

,isbn13,isbn10,title,authors,categories,thumbnail,description,published_year,average_rating,num_pages,...,fear_x,joy_x,sadness_x,surprise_x,anger_y,disgust_y,fear_y,joy_y,sadness_y,surprise_y
0,9780002005883,0002005883,Gilead,Marilynne Robinson,Fiction,http://books.google.com/books/content?id=KQZCP...,A NOVEL THAT READERS and critics have been eag...,2004.0,3.85,247.0,...,0.928169,0.932798,0.646215,0.967158,0.064134,0.273591,0.928169,0.932798,0.646215,0.967158
1,9780002261982,0002261987,Spider's Web,Charles Osborne;Agatha Christie,Detective and mystery stories,http://books.google.com/books/content?id=gA5GP...,A new 'Christie for Christmas' -- a full-lengt...,2000.0,3.83,241.0,...,0.942528,0.704422,0.887940,0.111690,0.612619,0.348287,0.942528,0.704422,0.887940,0.111690
2,9780006178736,0006178731,Rage of angels,Sidney Sheldon,Fiction,http://books.google.com/books/content?id=FKo2T...,"A memorable, mesmerizing heroine Jennifer -- b...",1993.0,3.93,512.0,...,0.972321,0.767238,0.549476,0.111690,0.064134,0.104007,0.972321,0.767238,0.549476,0.111690
3,9780006280897,0006280897,The Four Loves,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=XhQ5X...,Lewis' work on the nature of love divides love...,2002.0,4.15,170.0,...,0.360706,0.251883,0.732685,0.111690,0.351484,0.150723,0.360706,0.251883,0.732685,0.111690
4,9780006280934,0006280935,The Problem of Pain,Clive Staples Lewis,Christian life,http://books.google.com/books/content?id=Kk-uV...,"""In The Problem of Pain, C.S. Lewis, one of th...",2002.0,4.09,176.0,...,0.095043,0.040565,0.884389,0.475881,0.081412,0.184496,0.095043,0.040565,0.884389,0.475881
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5192,9788172235222,8172235224,Mistaken Identity,Nayantara Sahgal,Indic fiction (English),http://books.google.com/books/content?id=q-tKP...,On A Train Journey Home To North India After L...,2003.0,2.93,324.0,...,0.919165,0.255170,0.853721,0.980877,0.148209,0.030643,0.919165,0.255170,0.853721,0.980877
5193,9788173031014,8173031010,Journey to the East,Hermann Hesse,Adventure stories,http://books.google.com/books/content?id=rq6JP...,This book tells the tale of a man who goes on ...,2002.0,3.70,175.0,...,0.051363,0.400263,0.883198,0.111690,0.064134,0.114383,0.051363,0.400263,0.883198,0.111690
5194,9788179921623,817992162X,The Monk Who Sold His Ferrari: A Fable About F...,Robin Sharma,Health & Fitness,http://books.google.com/books/content?id=c_7mf...,"Wisdom to Create a Life of Passion, Purpose, a...",2003.0,3.82,198.0,...,0.339217,0.947779,0.375754,0.066685,0.009997,0.009929,0.339217,0.947779,0.375754,0.066685
5195,9788185300535,8185300534,I Am that,Sri Nisargadatta Maharaj;Sudhakar S. Dikshit,Philosophy,http://books.google.com/books/content?id=Fv_JP...,This collection of the timeless teachings of o...,1999.0,4.51,531.0,...,0.459270,0.759456,0.951104,0.368111,0.064134,0.104007,0.459270,0.759456,0.951104,0.368111


In [24]:
## Save the file
books.to_csv("books_with_emotions.csv")